# Input/Output

In [1]:
# imports

import os
from dotenv import load_dotenv

import openai
import anthropic
import google.generativeai

from IPython.display import Markdown, display, update_display

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC_


In [2]:
import gradio as gr 

In [3]:
# Let's wrap a call to GPT-4o-mini in a simple function

system_message = "You are a helpful assistant responding in Markdown."


def message_gpt(prompt,):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content
  

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

claude = anthropic.Anthropic()
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response


# The API for Gemini has a slightly different structure.
def stream_gemini(prompt):
    google.generativeai.configure() # if you don't run this, you will get an error
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_message
    )
    response = gemini.generate_content(prompt, stream=True)
    result = ""
    for chunk in response:
        result += chunk.text
        yield result
        
        
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result
    


In [4]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
import gradio as gr

system_message = "You are a helpful assistant responding in Markdown."

# Streaming functions for each model
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    for chunk in stream:
        yield chunk.choices[0].delta.content or ""

def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    with result as stream:
        for text in stream.text_stream:
            yield text or ""

def stream_gemini(prompt):
    google.generativeai.configure()  # Ensure proper configuration
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_message
    )
    response = gemini.generate_content(prompt, stream=True)
    for chunk in response:
        yield chunk.text

# Unified function to stream each model's output
def run_models_stream(prompt, selected_models):
    streams = {}
    if "GPT" in selected_models:
        streams["GPT"] = stream_gpt(prompt)
    if "Claude" in selected_models:
        streams["Claude"] = stream_claude(prompt)
    if "Gemini" in selected_models:
        streams["Gemini"] = stream_gemini(prompt)
    return streams

# Gradio Interface
def stream_responses(prompt, selected_models):
    streams = run_models_stream(prompt, selected_models)
    responses = {model: f"### {model} Response:\n" for model in selected_models}  # Add headers for each model

    while True:
        any_active = False
        for model, stream in streams.items():
            try:
                chunk = next(stream)
                responses[model] += chunk  # Append only new text
                yield [
                    responses.get("GPT", ""), 
                    responses.get("Claude", ""), 
                    responses.get("Gemini", "")
                ]
                any_active = True
            except StopIteration:
                continue
        if not any_active:
            break

# Gradio Layout
with gr.Blocks() as view:
    gr.Markdown("## Multi-Model Chat Interface with Streaming")
    
    with gr.Row():
        prompt_input = gr.Textbox(label="Your Message:")
    
    with gr.Row():
        model_selection = gr.CheckboxGroup(
            ["GPT", "Claude", "Gemini"],
            label="Select Models",
            value=["GPT"]
        )
    
    with gr.Row():
        gpt_output = gr.Markdown(label="GPT Response:")
        claude_output = gr.Markdown(label="Claude Response:")
        gemini_output = gr.Markdown(label="Gemini Response:")
    
    submit_btn = gr.Button("Submit")
    submit_btn.click(
        stream_responses,
        inputs=[prompt_input, model_selection],
        outputs=[gpt_output, claude_output, gemini_output],
        api_name="stream_responses"
    )

view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# Chatbot

In [6]:
# imports

import os
from dotenv import load_dotenv
import gradio as gr
import openai
import anthropic
import google.generativeai

from IPython.display import Markdown, display, update_display

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC_


In [7]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!
import ollama
system_message = "You are a helpful assistant responding in Markdown."
MODEL_LLAMA = 'phi'

def chat_ollama(message, history = []):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # stream = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages, stream=True)
    stream = ollama.chat(model=MODEL_LLAMA, messages=messages, stream = True)

    response = ""
    for chunk in stream:
        response += chunk
        yield response


def chat(message, history):
    openai_via_ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = openai_via_ollama.chat.completions.create(model=MODEL_LLAMA, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


In [8]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1602, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 710, in async_iteration
    return await anext(it